## First load the 2 datasets

In [0]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
!pip install apyori
import pandas as pd
from apyori import apriori
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#load train data
file_id = '1OOCbSHQjv4atpwjTFgpaGGxbaE7ofzqr'
downloaded = drive.CreateFile({'id':file_id}) 
downloaded.GetContentFile('TRAIN-ARULES.csv')  
data_in = pd.read_csv('TRAIN-ARULES.csv')
data_in.head()

#load test data
file_id = '1s9p4IB8CzMur1Iey4a4WdQjHjpfee8Ix'
downloaded = drive.CreateFile({'id':file_id}) 
downloaded.GetContentFile('testarules.csv')  
test_in = pd.read_csv('testarules.csv')
test_in.head()


,Item1,Item2,Item3,Item4,Item5
0,Dark Chocolate Minis,Organic Pink Lemonade Bunny Fruit Snacks,Peach-Pear Sparkling Water,NaN,NaN


## Group the records by order id

In [0]:
grouped_id_data = data_in.groupby(('order_id','user_id'))
result = gb['product_name'].unique()
print(result)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  """Entry point for launching an IPython kernel.


order_id  user_id
1483      90         [Organic Pink Lemonade Bunny Fruit Snacks, Dark Chocolate Minis, Sparkling Water, Natural Mango Essenced, Peach-Pear Sparkling Water, Organic Heritage Flakes Cereal, Popped Salted Caramel Granola ...
4595      66                                                                   [Creme De Menthe Thins, Milk Chocolate English Toffee Miniatures Candy Bars, Baker's Pure Cane Ultrafine Sugar, Plain Bagels, Cinnamon Bread]
7099      27         [Revive Zero Vitamin Water, VitaminWater Zero™ XXX Acai Blueberry Pomegranate, Vitamin Water Zero Rise Orange, Energy Tropical Citrus Vitamin Water Drink, Zero Vitamin Water, Vitamin Water Zero Sq...
8382      23         [Mountain Spring Water, Clementines, Bag, Pineapple on the Bottom Greek Yogurt, Pomegranate on the Bottom Non-Fat Greek Yogurt, Original French Vanilla Yogurt, Original Lowfat Lemon Burst Yogurt, ...
14400     36                                                                                      

In [0]:
grouped_id_data = data_in.groupby(('order_id','user_id'))
data = grouped_id_data['product_name'].unique()
print(data)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  """Entry point for launching an IPython kernel.


order_id  user_id
1483      90         [Organic Pink Lemonade Bunny Fruit Snacks, Dar...
4595      66         [Creme De Menthe Thins, Milk Chocolate English...
7099      27         [Revive Zero Vitamin Water, VitaminWater Zero™...
8382      23         [Mountain Spring Water, Clementines, Bag, Pine...
14400     36         [Truffle, Sea Salt, Black Truffle, 1500 Pale Ale]
19190     42         [Wild Albacore Tuna No Salt Added, Thin & Cris...
19256     13         [Half & Half, Whole Wheat Pita Bread Loaves, L...
20042     75         [Bag of Organic Bananas, English Seedless Cucu...
21889     71         [Authentic French Brioche, Petit Suisse Fruit,...
23391     7          [85% Lean Ground Beef, Organic Apple Slices, A...
29758     67         [Bag of Organic Bananas, Soda, Sparkling Miner...
29902     70         [Cherry Nighttime Instant Teething Pain Relief...
31801     46         [Chubby Hubby Ice Cream, Krinkle Cut Salt & Fr...
32664     90         [Organic Heritage Flakes Cereal, Squar

## Generate a list of the records

In [0]:
records = []

for i in range(data.shape[0]):
  records.append(data[i])

print(records)


[array(['Organic Pink Lemonade Bunny Fruit Snacks', 'Dark Chocolate Minis',
       'Sparkling Water, Natural Mango Essenced',
       'Peach-Pear Sparkling Water', 'Organic Heritage Flakes Cereal',
       'Popped Salted Caramel Granola Bars',
       'Healthy Grains Granola Bar, Vanilla Blueberry',
       'Flax Plus Organic Pumpkin Flax Granola',
       'Sweet & Salty Nut Almond Granola Bars',
       'Cool Mint Chocolate Energy Bar', 'Chocolate Chip Energy Bars',
       'Trail Mix Fruit & Nut Chewy Granola Bars'], dtype=object), array(['Creme De Menthe Thins',
       'Milk Chocolate English Toffee Miniatures Candy Bars',
       "Baker's Pure Cane Ultrafine Sugar", 'Plain Bagels',
       'Cinnamon Bread'], dtype=object), array(['Revive Zero Vitamin Water',
       'VitaminWater Zero™ XXX Acai Blueberry Pomegranate',
       'Vitamin Water Zero Rise Orange',
       'Energy Tropical Citrus Vitamin Water Drink', 'Zero Vitamin Water',
       'Vitamin Water Zero Squeezed Lemonade',
       'Total

## Generate association rules

In [0]:
print(test_in.shape)
checklist = [[]]
checklist.clear()
rowlist = []
for i in range(test_in.shape[0]):
  for j in range(test_in.shape[1]):
    if(pd.isnull(test_in.iloc[i,j]) != True):
      rowlist.append(test_in.iloc[i,j])
  if(rowlist != []):
    if(checklist.count != 0):
      checklist.append(rowlist)
    else:
      checklist=rowlist
    rowlist = []

print(checklist)


(1, 5)
[['Dark Chocolate Minis', 'Organic Pink Lemonade Bunny Fruit Snacks', 'Peach-Pear Sparkling Water']]


In [0]:
te = TransactionEncoder()
te_ary = te.fit(records).transform(records)
df = pd.DataFrame(te_ary, columns=te.columns_)
frequent_itemsets = apriori(df, min_support=0.0045, use_colnames=True)

frequent_itemsets

,support,itemsets
0,0.009873,(0% Greek Strained Yogurt)
1,0.004937,"(100% Juice, Variety Pack)"
2,0.009168,(100% Premium Select Not From Concentrate Pure...
3,0.005642,(100% Recycled Paper Towels)
4,0.014810,(1500 Pale Ale)
5,0.007757,(2% Reduced Fat Milk)
6,0.009168,(49 Flavors Jelly Belly Jelly Beans)
7,0.009168,(80 Vodka Holiday Edition)
8,0.008463,(85% Lean Ground Beef)
9,0.004937,(Acai Berry Chia Bar)


In [0]:
from mlxtend.frequent_patterns import association_rules

rules=association_rules(frequent_itemsets, metric="confidence", min_threshold=0.2)

## Predict items from the test data

In [0]:
pd.set_option('max_colwidth', 200)
for k in range(len(checklist)):
  print('\x1b[1;33m'+"Test Data[{}]={}".format(k,checklist[k])+'\x1b[0m')
  lst=checklist[k]
  res_dct = {lst[i]for i in range(0, len(lst))} 
  ans = rules[rules['antecedents'] == res_dct]
  print("Prediction:\n{}".format(ans['consequents']))

Test Data[0]=['Dark Chocolate Minis', 'Organic Pink Lemonade Bunny Fruit Snacks', 'Peach-Pear Sparkling Water']
Prediction:
3701                                       (Maple Pumpkin Seeds with Sea Salt Chewy with a Crunch Granola Bars)
3753                                                                             (Organic Graham Crunch Cereal)
3797                                                                           (Organic Heritage Flakes Cereal)
3838                                                                  (Sparkling Water, Natural Mango Essenced)
4912       (Organic Heritage Flakes Cereal, Maple Pumpkin Seeds with Sea Salt Chewy with a Crunch Granola Bars)
4943                                             (Organic Heritage Flakes Cereal, Organic Graham Crunch Cereal)
5028                                    (Sparkling Water, Natural Mango Essenced, Organic Graham Crunch Cereal)
5058                                  (Organic Heritage Flakes Cereal, Sparkling Water, Natu